In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

# Imbalanced learning package
from imblearn.ensemble import EasyEnsembleClassifier
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.over_sampling import ADASYN
from imblearn.pipeline import Pipeline

Here we import the bird strike data, and drop a number of columns that will be unused in this analysis.

In [2]:
df = pd.read_csv("wildlife.csv",parse_dates = ['INCIDENT_DATE '])
df_copy=df.copy().drop(columns=["TYPE_ENG","AMA","AMO","EMA","EMO","AC_CLASS","AC_MASS","REG","REMAINS_COLLECTED","REMAINS_SENT","RUNWAY","STR_RAD","DAM_RAD","STR_WINDSHLD","STR_NOSE","DAM_NOSE","STR_ENG1","DAM_ENG1","STR_ENG2","DAM_ENG2","STR_ENG3","DAM_ENG3","STR_ENG4","DAM_ENG4","STR_PROP","DAM_PROP","STR_WING_ROT","DAM_WING_ROT","STR_FUSE","DAM_FUSE","STR_LG","DAM_LG","STR_TAIL","DAM_TAIL","STR_LGHTS","DAM_LGHTS","STR_OTHER","DAM_OTHER","OTHER_SPECIFY","REPORTED_NAME","REPORTED_TITLE","TRANSFER", "Unnamed: 101"])

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (14,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Now we compile an (incomplete) list of species that are small, medium, and large, as we will use that as a factor in predicting damage or the severity of damage. These are one-hot encoded and stored. Unlisted species are treated as unknown size, and form the class that is dropped in the encoding. Addition of further species to these lists could potentially be helpful.

In [3]:
smalls = ['Barn swallow','Horned lark','Cliff swallow','Sparrows','Unknown bird - small']
df_copy['small'] = df_copy.SPECIES.isin(smalls).astype(int)

In [4]:
meds = ['Unknown bird - medium','Mourning dove','Killdeer',
        'American kestrel','European starling','Rock pigeon',
        'Eastern meadowlark','Western meadowlark','Ring-billed gull','American robin','Barn owl']
df_copy['medium'] = df_copy.SPECIES.isin(meds).astype(int)

In [5]:
larges = ['Unknown bird - large','Gulls','Red-tailed hawk']
df_copy['large'] = df_copy.SPECIES.isin(larges).astype(int)

As discovered in other analyses, a large portion of strikes occur on approach to landing, and thus we include this as as input with the hope that it can help predict the outcomes.

In [6]:
df_copy['on_approach'] = df_copy.PHASE_OF_FLIGHT.isin(['Approach']).astype(int)

We also consider the height at which the strike occurred. There are a number of NA values, which we replace with 0 to allow for numerical data analysis here.

In [7]:
df_copy.HEIGHT = df_copy.HEIGHT.fillna(0)

We also want to consider the weather, which is already one-hot encoded in the various PRECIP columns. In addition, other analyses showed that if bird was sucked into an engine, more monetary damage was likely, meaning we also include INGESTED in our input variables.

Now we one hot encode the two variables we hope to predict: Whether the strike resulted in damage, as well as if the damage was substantial or worse. The 'Class' designations are military designations, and were matched based on estimated value.

In [8]:
df_copy['damaged'] = df_copy.DAMAGE.isin(['S','D','M','M?','Class E','Class C','Class B']).astype(int)

In [9]:
print('The proportion of strikes that resulted in damage is:',sum(df_copy.damaged)/len(df_copy))

The proportion of strikes that resulted in damage is: 0.06256164576581655


Notice that the proportion of stikes that result in damage is very small, meaning that always predicting that there is no strike gives 93% accuracy. However, this is not helpful--we want to be able to predict a nonzero amount of damaging strikes without losing accuracy.

There is sufficient data to have a generous amount of training data in the train-test split, though we are careful to stratify.

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    df_copy[['small','medium','large','on_approach','HEIGHT','PRECIP_SNOW','PRECIP_RAIN','PRECIP_FOG','INGESTED']],
    df_copy['damaged'], test_size=0.1, random_state=440, stratify = df_copy['damaged'])

The imbalance in size makes basic classifiers insufficient in predicting damage from strikes, as shown by this fairly large fully connected neural net:

In [11]:
mlp = MLPClassifier(hidden_layer_sizes=(20,20,20,20,20),max_iter=4000)
mlp.fit(X_train,y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(20, 20, 20, 20, 20), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=4000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [12]:
pred_train = mlp.predict(X_train)
pred_test = mlp.predict(X_test)

print('Train accuracy:', sum(pred_train ==y_train)/len(y_train))
print('Test accuracy:', sum(pred_test ==y_test)/len(y_test))
print('True positives in training:', sum((pred_train==1) & (y_train))/sum(y_train))
print('True positives in testing:', sum((pred_test==1) & (y_test))/sum(y_test))
print('True negatives in training:', sum((pred_train==0) & (1-y_train))/sum(1-y_train))
print('True negatives in testing:', sum((pred_test==0) & (1-y_test))/sum(1-y_test))

Train accuracy: 0.9391138632302368
Test accuracy: 0.9385871149548148
True positives in training: 0.07706247842595788
True positives in testing: 0.06599378881987578
True negatives in training: 0.9966425366988592
True negatives in testing: 0.9968383953560692


This fairly large network is essentially a constant classfier, predicting very few instances of damage, without great accuracy. The network is unable to find any way to meaningfully distingush the classes, since it mostly only learns about the strikes with no damage.

To address the small proportion of strikes, we can use a balaced version of bagging to undersample the non-damaging strikes and raise the importance of the damaging strikes in a manner analogous to ADABoost. This will increase the numebr of true positives at the cost of accuracy.

In [13]:
bal_bag = BalancedBaggingClassifier(DecisionTreeClassifier(max_depth = 20) , n_estimators = 50,
                                   bootstrap = True, max_samples = 2000)

bal_bag.fit(X_train, y_train)

BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                                class_weight=None,
                                                                criterion='gini',
                                                                max_depth=20,
                                                                max_features=None,
                                                                max_leaf_nodes=None,
                                                                min_impurity_decrease=0.0,
                                                                min_impurity_split=None,
                                                                min_samples_leaf=1,
                                                                min_samples_split=2,
                                                                min_weight_fraction_leaf=0.0,
                                                                pre

In [14]:
pred_train = bal_bag.predict(X_train)
pred_test = bal_bag.predict(X_test)

print('Train accuracy:', sum(pred_train ==y_train)/len(y_train))
print('Test accuracy:', sum(pred_test ==y_test)/len(y_test))
print('True positives in training:', sum((pred_train==1) & (y_train))/sum(y_train))
print('True positives in testing:', sum((pred_test==1) & (y_test))/sum(y_test))
print('True negatives in training:', sum((pred_train==0) & (1-y_train))/sum(1-y_train))
print('True negatives in testing:', sum((pred_test==0) & (1-y_test))/sum(1-y_test))

Train accuracy: 0.7904778357834272
Test accuracy: 0.7916140316781654
True positives in training: 0.6517086641353124
True positives in testing: 0.6583850931677019
True negatives in training: 0.7997385440242336
True negatives in testing: 0.800507929926402


The EasyEnsembleClassifier is a built-in method for doing this kind of undersampling, and perfoms similarly:

In [15]:
model = EasyEnsembleClassifier(n_estimators=50)

model.fit(X_train, y_train)

EasyEnsembleClassifier(base_estimator=None, n_estimators=50, n_jobs=None,
                       random_state=None, replacement=False,
                       sampling_strategy='auto', verbose=0, warm_start=False)

In [16]:
pred_train = model.predict(X_train)
pred_test = model.predict(X_test)

print('Train accuracy:', sum(pred_train ==y_train)/len(y_train))
print('Test accuracy:', sum(pred_test ==y_test)/len(y_test))
print('True positives in training:', sum((pred_train==1) & (y_train))/sum(y_train))
print('True positives in testing:', sum((pred_test==1) & (y_test))/sum(y_test))
print('True negatives in training:', sum((pred_train==0) & (1-y_train))/sum(1-y_train))
print('True negatives in testing:', sum((pred_test==0) & (1-y_test))/sum(1-y_test))

Train accuracy: 0.7951908697788167
Test accuracy: 0.796861335147216
True positives in training: 0.6474801518812565
True positives in testing: 0.6506211180124224
True negatives in training: 0.805048288730326
True negatives in testing: 0.8066238208769566


Instead of undersampling, we can also try balancing the classes by oversampling using the ADASYN method, based on ADABoost. We will use the same size neural net as before, but now we have a much better rate of true positives:

In [17]:
mlp_pipe = Pipeline([('adasyn', ADASYN(random_state=440)),('mlp', MLPClassifier(hidden_layer_sizes=(10,10,10),max_iter=1000))])
#mlp_pipe = Pipeline([('adasyn', ADASYN()),('mlp', MLPClassifier(hidden_layer_sizes=(10,10,10),max_iter=1000))])

In [18]:
mlp_pipe.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('adasyn',
                 ADASYN(n_jobs=None, n_neighbors=5, random_state=440,
                        sampling_strategy='auto')),
                ('mlp',
                 MLPClassifier(activation='relu', alpha=0.0001,
                               batch_size='auto', beta_1=0.9, beta_2=0.999,
                               early_stopping=False, epsilon=1e-08,
                               hidden_layer_sizes=(10, 10, 10),
                               learning_rate='constant',
                               learning_rate_init=0.001, max_fun=15000,
                               max_iter=1000, momentum=0.9, n_iter_no_change=10,
                               nesterovs_momentum=True, power_t=0.5,
                               random_state=None, shuffle=True, solver='adam',
                               tol=0.0001, validation_fraction=0.1,
                               verbose=False, warm_start=False))],
         verbose=False)

In [19]:
pred_train = mlp_pipe.predict(X_train)
pred_test = mlp_pipe.predict(X_test)

print('Train accuracy:', sum(pred_train ==y_train)/len(y_train))
print('Test accuracy:', sum(pred_test ==y_test)/len(y_test))
print('True positives in training:', sum((pred_train==1) & (y_train))/sum(y_train))
print('True positives in testing:', sum((pred_test==1) & (y_test))/sum(y_test))
print('True negatives in training:', sum((pred_train==0) & (1-y_train))/sum(1-y_train))
print('True negatives in testing:', sum((pred_test==0) & (1-y_test))/sum(1-y_test))

Train accuracy: 0.7587822772646048
Test accuracy: 0.7610533475852687
True positives in training: 0.6811356575768036
True positives in testing: 0.6824534161490683
True negatives in training: 0.7639639950933812
True negatives in testing: 0.7663004042707577


An alternate way to get around this tradeoff is to change gears a bit, and only examine the strikes that caused damage in an attempt to predict when the damage is severe. Notice that this is a much larger proportion, though still small enough to need to stratify.

In [20]:
df_copy['high_damage'] = df_copy.DAMAGE.isin(['S','D','Class C','Class B']).astype(int)

In [21]:
print('The proportion of damaging strikes that resulted in high damage is:', sum(df_copy.high_damage)/sum(df_copy.damaged))

The proportion of damaging strikes that resulted in high damage is: 0.21186703945324634


Now we restrict our dataset to only these entries, and create a stratified train-test split.

In [22]:
df_damage = df_copy.copy()[df_copy.damaged > 0]
X_train, X_test, y_train, y_test = train_test_split(
    df_damage[['small','medium','large','on_approach','HEIGHT','PRECIP_SNOW','PRECIP_RAIN','PRECIP_FOG','INGESTED']],
    df_damage['high_damage'], test_size=0.1, random_state=440, stratify = df_damage['high_damage'])

Now we can use various classifiers here to attempt to classify the severity of the damage. Note that the MLP classifier is still mostly unable to exceed a fixed guess, but tree and forest classifiers can do better, likely due to the categorical nature of the input variables.

In [23]:
mlp = MLPClassifier(hidden_layer_sizes=(50,50,50,50,50,50,50,50),max_iter=10000)
mlp.fit(X_train,y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(50, 50, 50, 50, 50, 50, 50, 50),
              learning_rate='constant', learning_rate_init=0.001, max_fun=15000,
              max_iter=10000, momentum=0.9, n_iter_no_change=10,
              nesterovs_momentum=True, power_t=0.5, random_state=None,
              shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
              verbose=False, warm_start=False)

In [24]:
pred_train = mlp.predict(X_train)
pred_test = mlp.predict(X_test)

print('Train accuracy:', sum(pred_train ==y_train)/len(y_train))
print('Test accuracy:', sum(pred_test ==y_test)/len(y_test))
print('True positives in training:', sum((pred_train==1) & (y_train))/sum(y_train))
print('True positives in testing:', sum((pred_test==1) & (y_test))/sum(y_test))
print('True negatives in training:', sum((pred_train==0) & (1-y_train))/sum(1-y_train))
print('True negatives in testing:', sum((pred_test==0) & (1-y_test))/sum(1-y_test))

Train accuracy: 0.7880566102865033
Test accuracy: 0.781832298136646
True positives in training: 0.2920570264765784
True positives in testing: 0.31868131868131866
True negatives in training: 0.9213839921165006
True negatives in testing: 0.9064039408866995


In [25]:
tree = DecisionTreeClassifier(max_depth=20)
tree.fit(X_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=20, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [26]:
pred_train = tree.predict(X_train)
pred_test = tree.predict(X_test)

print('Train accuracy:', sum(pred_train ==y_train)/len(y_train))
print('Test accuracy:', sum(pred_test ==y_test)/len(y_test))
print('True positives in training:', sum((pred_train==1) & (y_train))/sum(y_train))
print('True positives in testing:', sum((pred_test==1) & (y_test))/sum(y_test))
print('True negatives in training:', sum((pred_train==0) & (1-y_train))/sum(1-y_train))
print('True negatives in testing:', sum((pred_test==0) & (1-y_test))/sum(1-y_test))

Train accuracy: 0.8187780462547463
Test accuracy: 0.8051242236024845
True positives in training: 0.23136456211812628
True positives in testing: 0.20146520146520147
True negatives in training: 0.9766779809482098
True negatives in testing: 0.967487684729064


In [27]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(max_depth = 10, n_estimators = 100)
forest.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [28]:
pred_train = forest.predict(X_train)
pred_test = forest.predict(X_test)

print('Train accuracy:', sum(pred_train ==y_train)/len(y_train))
print('Test accuracy:', sum(pred_test ==y_test)/len(y_test))
print('True positives in training:', sum((pred_train==1) & (y_train))/sum(y_train))
print('True positives in testing:', sum((pred_test==1) & (y_test))/sum(y_test))
print('True negatives in training:', sum((pred_train==0) & (1-y_train))/sum(1-y_train))
print('True negatives in testing:', sum((pred_test==0) & (1-y_test))/sum(1-y_test))

Train accuracy: 0.8107525025888851
Test accuracy: 0.8012422360248447
True positives in training: 0.21221995926680245
True positives in testing: 0.19047619047619047
True negatives in training: 0.9716413007774006
True negatives in testing: 0.9655172413793104


We can attempt to boost this with similar techniques, including a balanced bagging approach:

In [29]:
bal_bag = BalancedBaggingClassifier(DecisionTreeClassifier(max_depth = 10) , n_estimators = 50,
                                   bootstrap = True, max_samples = 2000)

bal_bag.fit(X_train, y_train)

BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                                class_weight=None,
                                                                criterion='gini',
                                                                max_depth=10,
                                                                max_features=None,
                                                                max_leaf_nodes=None,
                                                                min_impurity_decrease=0.0,
                                                                min_impurity_split=None,
                                                                min_samples_leaf=1,
                                                                min_samples_split=2,
                                                                min_weight_fraction_leaf=0.0,
                                                                pre

In [30]:
pred_train = bal_bag.predict(X_train)
pred_test = bal_bag.predict(X_test)

print('Train accuracy:', sum(pred_train ==y_train)/len(y_train))
print('Test accuracy:', sum(pred_test ==y_test)/len(y_test))
print('True positives in training:', sum((pred_train==1) & (y_train))/sum(y_train))
print('True positives in testing:', sum((pred_test==1) & (y_test))/sum(y_test))
print('True negatives in training:', sum((pred_train==0) & (1-y_train))/sum(1-y_train))
print('True negatives in testing:', sum((pred_test==0) & (1-y_test))/sum(1-y_test))

Train accuracy: 0.7406800138073869
Test accuracy: 0.7336956521739131
True positives in training: 0.624847250509165
True positives in testing: 0.63003663003663
True negatives in training: 0.7718164896529071
True negatives in testing: 0.761576354679803


In this instance, we can do substantially better in true positives, although our overall accuracy is actually worse than a constant predictor. 